## Subsetting the NETCDF4 data by Specifying Lat/Lon Range

The ERA5 dataset we used came from two sources:
1. Sinica: level-based data with domain of 10 \~ 50N/100 \~ 140E (161x161, 0.25 degree interval)
2. NTU: MSLP data with domain of -20 \~ 60N/60 \~ 180E (321x481, 0.25 degree interval)

In order to make further analysis easier, we need to subset the MSLP data to fit the domain of level-based data. We refer to [this article](https://stackoverflow.com/questions/48683135/how-to-subset-a-netcdf-file-using-latlong-details-and-rewrite-to-a-new-nc-file) to do the job.


### Test Ground for Subsetting

In [1]:
# Development test
from netCDF4 import Dataset
import numpy as np

lon0 = 100.
lon1 = 140.
lat0 = 10.
lat1 = 50.


nch500 = Dataset('../data/era5/h500/2018123100_h500.nc','r')
ncmslp = Dataset('D:\data\ERA5_nc\MSLP\MSLP_20181231.nc')

print(nch500)
print(ncmslp)

lons = ncmslp.variables['lon'][:]
lats = ncmslp.variables['lat'][:]

lat_lb = np.argmin(abs(lats-lat0))
lat_ub = np.argmin(abs(lats-lat1))+1

lon_lb = np.argmin(abs(lons-lon0))
lon_ub = np.argmin(abs(lons-lon1))+1

print(lat_lb, lat_ub, lon_lb, lon_ub)
print(lats[lat_lb], lats[lat_ub], lons[lon_lb], lons[lon_ub])

mslp = ncmslp.variables['psl'][0, lon_lb:lon_ub, lat_lb:lat_ub]
print(mslp.shape)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2021-03-30 15:42:51 GMT by grib_to_netcdf-2.18.0: grib_to_netcdf -o ../../../data/ERA5_nc//2018123100_h500.nc tmp.grb
    dimensions(sizes): longitude(161), latitude(161), time(1)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 z(time, latitude, longitude)
    groups: 
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: European Centre for Medium-Range Weather Forecasts
    history: 2021-04-10T17:07:51 GRIB to CDM+CF via cfgrib-0.9.8.4/ecCodes-2.16.0 with {"source": "/cache/data0/2b4cb15ef928098cd31ecd2f6b1e26c7.grib", "filter_by_keys": {}, "encode_cf": ["parameter", "time", "geography", "vertical"]}
    source: ECMWF
    dimensions(sizes): time(1), lat(321), lon(481)
    variables(dimensions): int64 time(time), int

### Function for single file conversion

In [2]:
def subset_era5_nc(infile, outfile, lat0=10., lat1=50., lon0=100., lon1=140., var='psl'):
    from netCDF4 import Dataset
    import numpy as np
    import time
    # Read input
    nc_file = Dataset(infile,'r')
    # Retrieve lons/lats
    lats = nc_file.variables['lat'][:]
    lons = nc_file.variables['lon'][:]
    # Calculat bounds
    lat_lb = np.argmin(abs(lats-lat0))
    lat_ub = np.argmin(abs(lats-lat1))+1
    lon_lb = np.argmin(abs(lons-lon0))
    lon_ub = np.argmin(abs(lons-lon1))+1
    # Subset the MLSP and lon/lat
    var_sub = nc_file.variables[var][0, lon_lb:lon_ub, lat_lb:lat_ub]
    var_units = nc_file.variables[var].units
    lat_sub = nc_file.variables['lat'][lat_lb:lat_ub]
    lon_sub = nc_file.variables['lon'][lon_lb:lon_ub]
    nc_file.close()

    # write to new file
    my_file = Dataset(outfile,'w', format = 'NETCDF4')
    my_file.discription = 'ERA5-MSLP-subset'
    my_file.history = 'Created on: ' +time.ctime(time.time())
    # Dimensions
    latdim = lat_ub-lat_lb # getting the no of element
    londim = lon_ub-lon_lb
    lon = my_file.createDimension('longitude',londim)
    lat = my_file.createDimension('lattitude',latdim)
    t = my_file.createDimension('time',1)
    # Variables
    latitude = my_file.createVariable('Lattitude',np.float32,('lattitude',))
    latitude.units = 'Degree_north'
    longitude = my_file.createVariable('Longitude',np.float32,('longitude',))
    longitude.units = 'Degree_east'
    ts = my_file.createVariable('time',np.int32,('time',))
    var = my_file.createVariable('slp',np.float32,('time','longitude','lattitude'),fill_value = -9999.0)
    var.units = var_units
    #Load values to the variables
    latitude[:] = lat_sub
    longitude[:] = lon_sub
    var[:] = var_sub
    my_file.close()
    
# Test the function
subset_era5_nc('D:\data\ERA5_nc\MSLP\MSLP_20181231.nc', '../data/era5/mslp/2018123100_mslp.nc')

In [3]:
# See test results
test = Dataset('../data/era5/mslp/2018123100_mslp.nc')
print(test)
print(test.variables['Lattitude'][:])
print(test.variables['Longitude'][:])
print(test.variables['slp'][:])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    discription: ERA5-MSLP-subset
    history: Created on: Thu May 20 10:44:08 2021
    dimensions(sizes): longitude(161), lattitude(161), time(1)
    variables(dimensions): float32 Lattitude(lattitude), float32 Longitude(longitude), int32 time(time), float32 slp(time, longitude, lattitude)
    groups: 
[10.   10.25 10.5  10.75 11.   11.25 11.5  11.75 12.   12.25 12.5  12.75
 13.   13.25 13.5  13.75 14.   14.25 14.5  14.75 15.   15.25 15.5  15.75
 16.   16.25 16.5  16.75 17.   17.25 17.5  17.75 18.   18.25 18.5  18.75
 19.   19.25 19.5  19.75 20.   20.25 20.5  20.75 21.   21.25 21.5  21.75
 22.   22.25 22.5  22.75 23.   23.25 23.5  23.75 24.   24.25 24.5  24.75
 25.   25.25 25.5  25.75 26.   26.25 26.5  26.75 27.   27.25 27.5  27.75
 28.   28.25 28.5  28.75 29.   29.25 29.5  29.75 30.   30.25 30.5  30.75
 31.   31.25 31.5  31.75 32.   32.25 32.5  32.75 33.   33.25 33.5  33.75
 34.   34.25 34.5  34.75 

### File collecting and Parsing

In [4]:
def list_era5_files(dir, prefix='MSLP_', suffix='.nc'):
    ''' To scan through the sapecified dir and get the corresponding file with suffix. '''
    import os
    import pandas as pd
    xfiles = []
    for root, dirs, files in os.walk(dir, followlinks=True):    # Loop through the directory
        for fn in files:
            if fn.endswith(suffix):                             # Filter files with suffix
                timestamp = fn.replace(suffix,'').replace(prefix,'')          # Removing the suffix to get time-stamp
                xfiles.append({'timestamp':timestamp, 'xuri':os.path.join(root, fn)})
    return(pd.DataFrame(xfiles).sort_values('timestamp').reset_index(drop=True))

# Test
infiles = list_era5_files('D:\data\ERA5_nc\MSLP')
print(infiles.head())

  timestamp                                   xuri
0  19790101  D:\data\ERA5_nc\MSLP\MSLP_19790101.nc
1  19790102  D:\data\ERA5_nc\MSLP\MSLP_19790102.nc
2  19790103  D:\data\ERA5_nc\MSLP\MSLP_19790103.nc
3  19790104  D:\data\ERA5_nc\MSLP\MSLP_19790104.nc
4  19790105  D:\data\ERA5_nc\MSLP\MSLP_19790105.nc


### Batch Processing

In [5]:
SRCPATH = 'D:\data\ERA5_nc\MSLP'
OUTPATH = '../data/era5/mslp/'
OUT_SUFFIX = '_mslp.nc'

#for i in range(100,110,2):
for i in range(infiles.shape[0]):
    infile = infiles.iloc[i]['xuri']
    outfile = OUTPATH + infiles.iloc[i]['timestamp'] + '00' + OUT_SUFFIX
    print(infile, outfile)
    subset_era5_nc(infile, outfile)
    

D:\data\ERA5_nc\MSLP\MSLP_19790101.nc ../data/era5/mslp/1979010100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790102.nc ../data/era5/mslp/1979010200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790103.nc ../data/era5/mslp/1979010300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790104.nc ../data/era5/mslp/1979010400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790105.nc ../data/era5/mslp/1979010500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790106.nc ../data/era5/mslp/1979010600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790107.nc ../data/era5/mslp/1979010700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790108.nc ../data/era5/mslp/1979010800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790109.nc ../data/era5/mslp/1979010900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790110.nc ../data/era5/mslp/1979011000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790111.nc ../data/era5/mslp/1979011100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790112.nc ../data/era5/mslp/1979011200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790113.nc ../data/era5/mslp/1979011300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

D:\data\ERA5_nc\MSLP\MSLP_19790502.nc ../data/era5/mslp/1979050200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790503.nc ../data/era5/mslp/1979050300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790504.nc ../data/era5/mslp/1979050400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790505.nc ../data/era5/mslp/1979050500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790506.nc ../data/era5/mslp/1979050600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790507.nc ../data/era5/mslp/1979050700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790508.nc ../data/era5/mslp/1979050800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790509.nc ../data/era5/mslp/1979050900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790510.nc ../data/era5/mslp/1979051000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790511.nc ../data/era5/mslp/1979051100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790512.nc ../data/era5/mslp/1979051200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790513.nc ../data/era5/mslp/1979051300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790514.nc ../data/era5/mslp/1979051400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

D:\data\ERA5_nc\MSLP\MSLP_19790831.nc ../data/era5/mslp/1979083100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790901.nc ../data/era5/mslp/1979090100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790902.nc ../data/era5/mslp/1979090200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790903.nc ../data/era5/mslp/1979090300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790904.nc ../data/era5/mslp/1979090400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790905.nc ../data/era5/mslp/1979090500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790906.nc ../data/era5/mslp/1979090600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790907.nc ../data/era5/mslp/1979090700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790908.nc ../data/era5/mslp/1979090800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790909.nc ../data/era5/mslp/1979090900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790910.nc ../data/era5/mslp/1979091000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790911.nc ../data/era5/mslp/1979091100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790912.nc ../data/era5/mslp/1979091200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

D:\data\ERA5_nc\MSLP\MSLP_19791228.nc ../data/era5/mslp/1979122800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791229.nc ../data/era5/mslp/1979122900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791230.nc ../data/era5/mslp/1979123000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791231.nc ../data/era5/mslp/1979123100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800101.nc ../data/era5/mslp/1980010100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800102.nc ../data/era5/mslp/1980010200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800103.nc ../data/era5/mslp/1980010300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800104.nc ../data/era5/mslp/1980010400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800105.nc ../data/era5/mslp/1980010500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800106.nc ../data/era5/mslp/1980010600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800107.nc ../data/era5/mslp/1980010700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800108.nc ../data/era5/mslp/1980010800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800109.nc ../data/era5/mslp/1980010900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

D:\data\ERA5_nc\MSLP\MSLP_19800425.nc ../data/era5/mslp/1980042500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800426.nc ../data/era5/mslp/1980042600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800427.nc ../data/era5/mslp/1980042700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800428.nc ../data/era5/mslp/1980042800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800429.nc ../data/era5/mslp/1980042900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800430.nc ../data/era5/mslp/1980043000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800501.nc ../data/era5/mslp/1980050100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800502.nc ../data/era5/mslp/1980050200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800503.nc ../data/era5/mslp/1980050300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800504.nc ../data/era5/mslp/1980050400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800505.nc ../data/era5/mslp/1980050500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800506.nc ../data/era5/mslp/1980050600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800507.nc ../data/era5/mslp/1980050700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

...

D:\data\ERA5_nc\MSLP\MSLP_20171001.nc ../data/era5/mslp/2017100100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171002.nc ../data/era5/mslp/2017100200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171003.nc ../data/era5/mslp/2017100300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171004.nc ../data/era5/mslp/2017100400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171005.nc ../data/era5/mslp/2017100500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171006.nc ../data/era5/mslp/2017100600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171007.nc ../data/era5/mslp/2017100700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171008.nc ../data/era5/mslp/2017100800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171009.nc ../data/era5/mslp/2017100900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171010.nc ../data/era5/mslp/2017101000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171011.nc ../data/era5/mslp/2017101100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171012.nc ../data/era5/mslp/2017101200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20171013.nc ../data/era5/mslp/2017101300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

D:\data\ERA5_nc\MSLP\MSLP_20180124.nc ../data/era5/mslp/2018012400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180125.nc ../data/era5/mslp/2018012500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180126.nc ../data/era5/mslp/2018012600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180127.nc ../data/era5/mslp/2018012700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180128.nc ../data/era5/mslp/2018012800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180129.nc ../data/era5/mslp/2018012900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180130.nc ../data/era5/mslp/2018013000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180131.nc ../data/era5/mslp/2018013100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180201.nc ../data/era5/mslp/2018020100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180202.nc ../data/era5/mslp/2018020200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180203.nc ../data/era5/mslp/2018020300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180204.nc ../data/era5/mslp/2018020400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180205.nc ../data/era5/mslp/2018020500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

D:\data\ERA5_nc\MSLP\MSLP_20180525.nc ../data/era5/mslp/2018052500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180526.nc ../data/era5/mslp/2018052600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180527.nc ../data/era5/mslp/2018052700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180528.nc ../data/era5/mslp/2018052800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180529.nc ../data/era5/mslp/2018052900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180530.nc ../data/era5/mslp/2018053000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180531.nc ../data/era5/mslp/2018053100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180601.nc ../data/era5/mslp/2018060100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180602.nc ../data/era5/mslp/2018060200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180603.nc ../data/era5/mslp/2018060300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180604.nc ../data/era5/mslp/2018060400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180605.nc ../data/era5/mslp/2018060500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180606.nc ../data/era5/mslp/2018060600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

D:\data\ERA5_nc\MSLP\MSLP_20180923.nc ../data/era5/mslp/2018092300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180924.nc ../data/era5/mslp/2018092400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180925.nc ../data/era5/mslp/2018092500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180926.nc ../data/era5/mslp/2018092600_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180927.nc ../data/era5/mslp/2018092700_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180928.nc ../data/era5/mslp/2018092800_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180929.nc ../data/era5/mslp/2018092900_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180930.nc ../data/era5/mslp/2018093000_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20181001.nc ../data/era5/mslp/2018100100_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20181002.nc ../data/era5/mslp/2018100200_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20181003.nc ../data/era5/mslp/2018100300_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20181004.nc ../data/era5/mslp/2018100400_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20181005.nc ../data/era5/mslp/2018100500_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP

PermissionError: [Errno 13] Permission denied: b'../data/era5/mslp/2018123100_mslp.nc'